In [1]:
import pandas as pd

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
from scripts import util

In [5]:
conn = util.connect_rds()

In [6]:
from gmplot import gmplot

In [7]:
q = 'select * from rws_schema.ongevallen_raw';

In [8]:
df = pd.read_sql(q, con=conn)

In [9]:
df.columns

Index(['id_jaar', 'ongeval_id', 'bron', 'datumtijd', 'uur', 'dag', 'maand',
       'jaar', 'bps_key', 'wvkid_begdat_hecto_key', 'rijrichting', 'coords_x',
       'coords_y', 'afloop', 'key', 'weg', 'hecto', 'hectomtrng', 'dvk_letter',
       'pos_tv_wol'],
      dtype='object')

In [10]:
def clean_euro_decimals(series):
    '''
    Cleans the euro , decimals (ugh) into point decimals
    and converts series into numeric
    '''
    return pd.to_numeric(series.str.replace(',','.'))

In [17]:
df['coords_x'] = pd.to_numeric(df.coords_x.str.replace(',', '.'))
df['coords_y'] = pd.to_numeric(df.coords_y.str.replace(',', '.'))

In [117]:
df['date'] = df.datumtijd.map(lambda x: x.date())

In [114]:
pd.DatetimeIndex(df.datumtijd)

DatetimeIndex(['2018-02-19 14:56:30', '2018-02-19 14:25:46',
               '2018-02-16 06:31:28', '2018-02-15 15:11:22',
               '2018-02-15 10:51:55', '2018-02-15 09:44:45',
               '2018-02-15 09:02:05', '2018-02-15 08:46:36',
               '2018-02-14 12:55:18', '2018-02-14 12:53:34',
               ...
               '2012-01-01 12:27:00', '2012-01-01 11:37:00',
               '2012-01-01 10:32:00', '2012-01-01 09:38:00',
               '2012-01-01 08:25:00', '2012-01-01 06:51:00',
               '2012-01-01 05:45:00', '2012-01-01 05:20:00',
               '2012-01-01 03:49:00', '2012-01-01 00:54:00'],
              dtype='datetime64[ns]', name='datumtijd', length=176112, freq=None)

In [210]:
df['datetime'] = pd.DatetimeIndex(df.datumtijd).round('60min')

In [211]:
df.head()

,id_jaar,ongeval_id,bron,datumtijd,uur,dag,maand,jaar,bps_key,wvkid_begdat_hecto_key,...,coords_y,afloop,key,weg,hecto,hectomtrng,dvk_letter,pos_tv_wol,datetime,date
0,90088.2018,90088,NDW,2018-02-19 14:56:30,14,19,2,2018,None,497347012.2017-11-01.608,...,52.241428,None,35.608.Li,35,"60,8",608,None,Li,2018-02-19 15:00:00,2018-02-19
1,90087.2018,90087,NDW,2018-02-19 14:25:46,14,19,2,2018,None,275279004.2017-09-01.795,...,51.941662,None,2.795.Li,2,"79,5",795,None,Li,2018-02-19 14:00:00,2018-02-19
2,90086.2018,90086,NDW,2018-02-16 06:31:28,6,16,2,2018,None,177303013.2017-12-01.99,...,52.051788,None,12.99.Re,12,"9,9",99,None,Re,2018-02-16 07:00:00,2018-02-16
3,90085.2018,90085,NDW,2018-02-15 15:11:22,15,15,2,2018,None,196267066.2014-11-01.236,...,51.884312,None,16.236.y,16,"23,6",236,y,Li,2018-02-15 15:00:00,2018-02-15
4,90084.2018,90084,NDW,2018-02-15 10:51:55,10,15,2,2018,None,216377018.2015-04-01.97,...,52.384201,None,200.97.c,200,"9,7",97,c,Li,2018-02-15 11:00:00,2018-02-15


In [212]:
df_sample = df.loc[((df.jaar=='2018') & ((df.maand=='2') | (df.maand=='1')))]

In [213]:
grouped_coords = []
for interval in df_sample.datetime.sort_values().unique():
    temp = df_sample.loc[df_sample.datetime == pd.to_datetime(interval)]
    group = list(zip(temp.coords_y, temp.coords_x))
    group = [list(x) for x in group]
    grouped_coords.append(group)

In [175]:
import folium
import datetime as dt
import random as rnd
from folium.plugins import HeatMap

t0 = dt.datetime.now()

PT_COORD = [52.0297195, 4.5003863]

# sample_coords = rnd.sample(list(coords),5000)

map_pt = folium.Map(location=PT_COORD, zoom_start=7.75,width=1000, height=1000, tiles='Stamen Terrain')

# Plot coordinates using comprehension list
# [folium.CircleMarker(sample_coords[i], radius=2,
#                 color='#0080bb', fill_color='#0080bb').add_to(map_pt)] 

HeatMap(grouped_coords[2]).add_to(map_pt)



In [176]:
map_pt

In [60]:
t = pd.date_range("1-1-2016", "31-12-2016", freq="30min")[0:1000]

In [75]:
[coords,t]

[[(51.508426863788, 5.273272266423049),
  (51.880564283289004, 5.35915981570258),
  (51.9510383606, 4.5311198235),
  (53.033921472196205, 6.57946453133526),
  (52.0724251633509, 5.03241651486872),
  (51.28934, 5.6439129999999995),
  (51.9512596130371, 6.05639982223511),
  (52.339328765869105, 4.774889945983889),
  (52.0646, 4.7958),
  (51.459011077880895, 3.6041901111602797),
  (51.2689, 6.0853),
  (51.3001, 5.6228),
  (52.21679, 5.217272),
  (52.732170195155895, 6.4647800057002),
  (51.64799, 4.704697),
  (51.35711055187739, 6.14384625741318),
  (52.05492295527121, 5.137350215563231),
  (53.1677090491911, 6.805394106242111),
  (51.5456550534017, 4.5163301148210095),
  (51.98363, 5.0913580000000005),
  (52.0064, 5.1164),
  (52.1108, 5.2799),
  (51.7190769415463, 5.33394661375759),
  (52.3114814758, 5.3283801079),
  (51.4189, 4.127568),
  (53.12235, 6.610411),
  (52.1577, 5.167999999999999),
  (51.7184405291187, 5.33498381601196),
  (52.2030913655352, 5.194765960911139),
  (51.936768433

In [223]:
len(grouped_coords)

693

In [236]:
tindex = df_sample.datetime.sort_values().unique()

In [237]:
tindex = [str(s) for s in tindex]

In [238]:
m = folium.Map(location=PT_COORD, tiles='Stamen Terrain', zoom_start=7.75)
plugins.HeatMapWithTime(data=grouped_coords,
                        index = tindex,
                        auto_play=True).add_to(m)


In [239]:
m

In [170]:
d

NameError: name 'd' is not defined

In [84]:


import folium
import folium.plugins as plugins
import numpy as np

np.random.seed(3141592)
initial_data = (
    np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
    np.array([[48, 5]])
)

move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]



In [182]:
grouped_coords[:2]

[[(52.241428375244105, 6.75935506820679),
  (51.9416618347168, 5.14758014678955),
  (52.051788330078104, 4.42396306991577),
  (51.8843116760254, 4.564720153808589),
  (52.38420104980471, 4.70029401779175),
  (51.690250396728494, 4.88486003875732),
  (51.5273590087891, 4.7682900428772),
  (51.642330169677706, 4.69430685043335),
  (51.708038330078104, 4.88861083984375),
  (52.997371673584006, 5.707366943359379),
  (51.948028564453104, 5.0123600959777805),
  (51.884059906005895, 4.97177982330322),
  (51.8975410461426, 4.548649311065669),
  (51.764808654785206, 5.27398014068604),
  (52.5031394958496, 4.92152214050293),
  (51.903968811035206, 5.45627593994141),
  (52.632961273193395, 4.71550798416138),
  (51.9245681762695, 4.35773801803589),
  (51.983089447021506, 4.3296217918396005),
  (52.007740020752, 4.381945133209229),
  (51.974479675293, 4.39826107025146),
  (51.732639312744105, 5.8478798866272),
  (52.0700607299805, 4.38644218444824),
  (51.8900299072266, 4.3780779838562),
  (51.9383

In [85]:
data

[[[47.504539036025584, 4.292407029209247],
  [47.69040904564529, 4.252424765823684],
  [46.56622363785289, 3.1698932994737725],
  [48.512345988527414, 3.5575115701151936],
  [47.912356200438545, 4.541688488218782],
  [47.7449114720645, 6.410987139161764],
  [49.27575216702049, 3.583654594233588],
  [47.82845322406549, 3.8219354061097293],
  [48.04493261439567, 6.31593606743332],
  [46.476548975848104, 4.6875193522227105],
  [48.22998111187808, 4.330735827816389],
  [49.09176755111292, 5.097273923224413],
  [48.27319149417738, 5.658018973640203],
  [48.122983644592075, 5.780950867261257],
  [46.44643679370932, 4.755224680829496],
  [49.29166945103235, 6.129019031044898],
  [47.001037365055055, 6.290837811317019],
  [46.8662712198302, 5.128384470003526],
  [48.60872159648292, 4.685247191774352],
  [48.847472424367105, 4.528416690851091],
  [49.82969363846115, 3.7616826932379714],
  [47.41209462975057, 5.286337622398916],
  [47.61135491854893, 4.894699684890315],
  [47.977061435131525, 5.

In [181]:
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(data)

hm.add_to(m)

m

In [87]:
from datetime import datetime, timedelta


time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(data))
]


In [33]:
time_index

['2018-06-22',
 '2018-06-23',
 '2018-06-24',
 '2018-06-25',
 '2018-06-26',
 '2018-06-27',
 '2018-06-28',
 '2018-06-29',
 '2018-06-30',
 '2018-07-01',
 '2018-07-02',
 '2018-07-03',
 '2018-07-04',
 '2018-07-05',
 '2018-07-06',
 '2018-07-07',
 '2018-07-08',
 '2018-07-09',
 '2018-07-10',
 '2018-07-11',
 '2018-07-12',
 '2018-07-13',
 '2018-07-14',
 '2018-07-15',
 '2018-07-16',
 '2018-07-17',
 '2018-07-18',
 '2018-07-19',
 '2018-07-20',
 '2018-07-21',
 '2018-07-22',
 '2018-07-23',
 '2018-07-24',
 '2018-07-25',
 '2018-07-26',
 '2018-07-27',
 '2018-07-28',
 '2018-07-29',
 '2018-07-30',
 '2018-07-31',
 '2018-08-01',
 '2018-08-02',
 '2018-08-03',
 '2018-08-04',
 '2018-08-05',
 '2018-08-06',
 '2018-08-07',
 '2018-08-08',
 '2018-08-09',
 '2018-08-10',
 '2018-08-11',
 '2018-08-12',
 '2018-08-13',
 '2018-08-14',
 '2018-08-15',
 '2018-08-16',
 '2018-08-17',
 '2018-08-18',
 '2018-08-19',
 '2018-08-20',
 '2018-08-21',
 '2018-08-22',
 '2018-08-23',
 '2018-08-24',
 '2018-08-25',
 '2018-08-26',
 '2018-08-

In [92]:
data[0][0]

[47.504539036025584, 4.292407029209247]

In [29]:
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(
    data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m